In [1]:
import collections
import numpy as np
import pandas as pd
import pickle
from plotly.offline import iplot
from surprise import SVD, KNNBasic, CoClustering
from surprise import Dataset
from surprise import Reader
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise import Dataset
from surprise import Reader

Nesse notebook foram aplicados 3 modelos de Machine learing usando Gridsearch para encontrar os melhores parametros e cross validation para realizar a comparação entre eles.

In [2]:
ratings_df = pd.read_csv("ratings.csv")
ratings_df =  ratings_df.drop(['timestamp'], axis=1)
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

In [10]:
param_grid = {'n_epochs': [20, 25, 30],
              'lr_all': [0.002, 0.005],
              'reg_all': [0.04, 0.06]}
gsSVD = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)

gsSVD.fit(data)

print(gsSVD.best_score['rmse'])
print(gsSVD.best_params['rmse'])
print(gsSVD.cv_results)

0.8700862573350433
{'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.06}
{'split0_test_rmse': array([0.89049194, 0.89091118, 0.87831318, 0.87827227, 0.88757083,
       0.8873722 , 0.87707724, 0.874464  , 0.88559662, 0.88477364,
       0.87550712, 0.87271369]), 'split1_test_rmse': array([0.88997935, 0.88934081, 0.87865847, 0.87635611, 0.88632404,
       0.88583647, 0.87509169, 0.87392765, 0.88359409, 0.88238605,
       0.87579012, 0.87302009]), 'split2_test_rmse': array([0.88327115, 0.88269478, 0.86989188, 0.86973919, 0.87941506,
       0.87901226, 0.86971451, 0.86713235, 0.87775134, 0.87697014,
       0.86476521, 0.86452499]), 'mean_test_rmse': array([0.88791415, 0.88764892, 0.87562118, 0.87478919, 0.88443664,
       0.88407365, 0.87396115, 0.87184133, 0.88231402, 0.88137661,
       0.87202082, 0.87008626]), 'std_test_rmse': array([0.00328976, 0.00356129, 0.00405368, 0.00365557, 0.00358709,
       0.00363344, 0.00311031, 0.00333695, 0.00332827, 0.00326475,
       0.00513179, 0.0039344 ]),

In [11]:
svd = SVD(**gsSVD.best_params['rmse'])
svd.fit(data.build_full_trainset())

In [5]:
ratings_df

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100831,610,166534,4.0
100832,610,168248,5.0
100833,610,168250,5.0
100834,610,168252,5.0


In [6]:
print(svd.predict(uid=1, iid=6).est)

4.641159724372494


In [3]:
param_grid = {'n_epochs': [20, 25, 30], 'n_cltr_u': [5, 10, 20], 'n_cltr_i': [5, 10, 15]}
gsCC = GridSearchCV(CoClustering, param_grid, measures=['rmse'], cv=3)

gsCC.fit(data)

print(gsCC.best_score['rmse'])
print(gsCC.best_params['rmse'])
print(gsCC.cv_results)

0.9569779141167153
{'n_epochs': 30, 'n_cltr_u': 5, 'n_cltr_i': 5}
{'split0_test_rmse': array([0.95371413, 0.95972831, 0.96336149, 0.95886592, 0.96699358,
       0.96838221, 0.9562673 , 0.97426108, 0.98420844, 0.95386088,
       0.95666863, 0.95971086, 0.95720803, 0.9660221 , 0.9710521 ,
       0.96540131, 0.97597559, 0.98513221, 0.95196727, 0.96217828,
       0.95820385, 0.95818535, 0.96849792, 0.97146309, 0.96015388,
       0.97769409, 0.98402454]), 'split1_test_rmse': array([0.96180924, 0.96508181, 0.97346058, 0.96559422, 0.97863194,
       0.97730375, 0.97443162, 0.9880279 , 0.99328253, 0.96293257,
       0.96847297, 0.97001948, 0.97062998, 0.97468139, 0.97686726,
       0.96946052, 0.98893911, 0.99151348, 0.96296195, 0.96844199,
       0.96707304, 0.9707514 , 0.97911265, 0.97876788, 0.9750185 ,
       0.98928303, 0.99709467]), 'split2_test_rmse': array([0.95654592, 0.96055736, 0.9645307 , 0.95940637, 0.98020654,
       0.97608871, 0.96961771, 0.97967898, 0.98472215, 0.96016503,
   

In [7]:
cc = CoClustering(**gsCC.best_params['rmse'])
cc.fit(data.build_full_trainset())


<ipython-input-7-f236c642ebb3>:2: DeprecationWarning:

`np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



In [9]:
param_grid = {'k': [20, 30, 40],
              'min_k': [1, 2, 5]}
gsKNN = GridSearchCV(KNNBasic, param_grid, measures=['rmse'], cv=3)

gsKNN.fit(data)

print(gsKNN.best_score['rmse'])
print(gsKNN.best_params['rmse'])
print(gsKNN.cv_results)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

In [30]:

# data = Dataset.load_builtin('ml-100k')
knn = KNNBasic(**gsKNN.best_params['rmse'])
knn.fit(data.build_full_trainset())

Computing the msd similarity matrix...
Done computing similarity matrix.


In [34]:
results_svd = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
results_cc = cross_validate(cc, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
results_knn = cross_validate(knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8594  0.8701  0.8587  0.8609  0.8588  0.8616  0.0043  
MAE (testset)     0.6628  0.6646  0.6586  0.6605  0.6593  0.6612  0.0022  
Fit time          6.70    6.70    6.81    6.85    6.78    6.77    0.06    
Test time         0.11    0.51    0.12    0.11    0.11    0.19    0.16    
Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9542  0.9510  0.9507  0.9445  0.9484  0.9497  0.0032  
MAE (testset)     0.7379  0.7351  0.7352  0.7306  0.7348  0.7347  0.0023  
Fit time          2.18    2.13    2.10    2.13    2.10    2.13    0.03    
Test time         0.27    0.08    0.08    0.07    0.27    0.16    0.10    
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing simila

In [40]:
compare_models = {'SVD': [results_svd['test_rmse'][-1],
results_svd['test_mae'][-1]], 'CoClustering': [results_cc['test_rmse'][-1],
results_cc['test_mae'][-1]], 'KNNBasic': [results_knn['test_rmse'][-1],
results_knn['test_mae'][-1]], }
compare_models

{'SVD': [0.8588014073596785, 0.6592670999614225],
 'CoClustering': [0.9483579758216114, 0.7348340828586255],
 'KNNBasic': [0.9336057324482449, 0.7147185915762925]}

In [42]:
compare_models_df = pd.DataFrame(compare_models, index=['RMSE', 'MAE'])
compare_models_df

,SVD,CoClustering,KNNBasic
RMSE,0.858801,0.948358,0.933606
MAE,0.659267,0.734834,0.714719


In [12]:
pickle.dump( svd, open( "model_selected.p", "wb" ) )